<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain5_chromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q chromaDB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.6 MB/s eta 0:0

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

ef= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name ="all-MiniLm-L6-V2"
)

In [3]:
client= chromadb.Client()
collection = client.create_collection(
    name='document',
    embedding_function=ef,
    metadata={"description": "Used to demo filtering in ChromaDB"}
)
print(f"created collections{collection}")

created collectionsCollection(name=document)


###adding docs

In [4]:
collection.add(
    documents=[
        "This is a document about LangChain",
        "This is a reading about LlamaIndex",
        "This is a book about Python",
        "This is a document about pandas",
        "This is another document about LangChain"
    ],
    metadatas=[
        {"source": "langchain.com", "version": 0.1},
        {"source": "llamaindex.ai", "version": 0.2},
        {"source": "python.org", "version": 0.3},
        {"source": "pandas.pydata.org", "version": 0.4},
        {"source": "langchain.com", "version": 0.5},
    ],
    ids=["id1", "id2", "id3", "id4", "id5"]
)

In [5]:
collection.get()

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex',
  'This is a book about Python',
  'This is a document about pandas',
  'This is another document about LangChain'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'version': 0.1, 'source': 'langchain.com'},
  {'version': 0.2, 'source': 'llamaindex.ai'},
  {'version': 0.3, 'source': 'python.org'},
  {'source': 'pandas.pydata.org', 'version': 0.4},
  {'version': 0.5, 'source': 'langchain.com'}]}

In [6]:
collection.get(
    where={"source": "langchain.com"}
)

{'ids': ['id1', 'id5'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is another document about LangChain'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.1},
  {'version': 0.5, 'source': 'langchain.com'}]}

In [7]:
collection.delete(
    where={"source": "python.org"},
    ids='id3'
)

In [8]:
collection.get()

{'ids': ['id1', 'id2', 'id4', 'id5'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex',
  'This is a document about pandas',
  'This is another document about LangChain'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'version': 0.1, 'source': 'langchain.com'},
  {'source': 'llamaindex.ai', 'version': 0.2},
  {'version': 0.4, 'source': 'pandas.pydata.org'},
  {'source': 'langchain.com', 'version': 0.5}]}

###chaining filters

In [9]:
collection.get(
    where={
        "$and":[
            {"source": {"$in": ["langchain.com", "llamaindex.ai"]}},
            {"version": {"$lte": 0.3}}
        ]
    }
)

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.1},
  {'source': 'llamaindex.ai', 'version': 0.2}]}

##filtering based on document content


In [10]:
collection.get(
    where_document={
        "$or":[
            {"$contains":"Python"},
            {"$contains":"pandas"}
        ]
    }
)

{'ids': ['id4'],
 'embeddings': None,
 'documents': ['This is a document about pandas'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'pandas.pydata.org', 'version': 0.4}]}

##combining meta data filter as well as document filters

In [11]:
collection.get(
     where={"version": {"$gt": 0.1}},
    where_document={
        "$or": [
            {"$contains": "LangChain"},
            {"$contains": "Python"}
        ]
    }
)

{'ids': ['id5'],
 'embeddings': None,
 'documents': ['This is another document about LangChain'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'version': 0.5, 'source': 'langchain.com'}]}

#HNSW - Hierarchical navigable small world


In [21]:
client = chromadb.Client()
collection = client.create_collection(
    name='ew_document',
    embedding_function=ef,
    metadata={'tpoic':'query testing'},
    configuration={
        "hnsw":{
            "space":"ip",
            "ef_search":100,
            "ef_construction":100,
            "max_neighbors":16
        }
    }
)

In [22]:
collection.add(
    documents=[
        "Giant pandas are a bear species that lives in mountainous areas.",
        "A pandas DataFrame stores two-dimensional, tabular data",
        "I think everyone agrees that pandas are some of the cutest animals on the planet",
        "A direct comparison between pandas and polars indicates that polars is a more efficient library than pandas.",
    ],
    metadatas=[
        {"topic": "animals"},
        {"topic": "data analysis"},
        {"topic": "animals"},
        {"topic": "data analysis"},
    ],
    ids=["id1", "id2", "id3", "id4"]
)

In [23]:
collection.query(
    query_texts=["cats"],
    n_results=10,
)


{'ids': [['id3', 'id1', 'id2', 'id4']],
 'embeddings': None,
 'documents': [['I think everyone agrees that pandas are some of the cutest animals on the planet',
   'Giant pandas are a bear species that lives in mountainous areas.',
   'A pandas DataFrame stores two-dimensional, tabular data',
   'A direct comparison between pandas and polars indicates that polars is a more efficient library than pandas.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'topic': 'animals'},
   {'topic': 'animals'},
   {'topic': 'data analysis'},
   {'topic': 'data analysis'}]],
 'distances': [[0.738014280796051,
   0.8351748585700989,
   0.8634340763092041,
   0.9299634695053101]]}

In [24]:
collection.query(
    query_texts=["polar bear"],
    n_results=1,
)

{'ids': [['id4']],
 'embeddings': None,
 'documents': [['A direct comparison between pandas and polars indicates that polars is a more efficient library than pandas.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'topic': 'data analysis'}]],
 'distances': [[0.624370276927948]]}

this fails as it refers to the python framework polar not the animal , this can be corrected by either changing the model or providing filters such as the metadata filter

In [25]:
collection.query(
    query_texts=["polar bear"],
    n_results=1,
    where={'topic': 'animals'}
)

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Giant pandas are a bear species that lives in mountainous areas.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'topic': 'animals'}]],
 'distances': [[0.7096826434135437]]}

You could use the document filter as well, it's suggested to use both metadata as well as document filtering

In [26]:
collection.query(
    query_texts=["polar bear"],
    n_results=1,
    where={'topic': 'animals'},
    where_document={'$not_contains': 'library'}
)

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Giant pandas are a bear species that lives in mountainous areas.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'topic': 'animals'}]],
 'distances': [[0.7096826434135437]]}